In [1]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

In [2]:
#Parameters
file = "../tables/2021AP.xlsx"
next_week = 5
#Read in df
df = pd.read_excel(file)
df

Team  Week  Rank  W  L  Winning Percentage  Opp. Rank  \
0            Alabama     1     1  0  0                   1         14   
1           Oklahoma     1     2  0  0                   1         26   
2            Clemson     1     3  0  0                   1          5   
3         Ohio State     1     4  0  0                   1         26   
4            Georgia     1     5  0  0                   1          3   
..               ...   ...   ... .. ..                 ...        ...   
146    Arizona State     6    22  4  1                 0.8        NaN   
147         NC State     6    23  4  1                 0.8        NaN   
148              SMU     6    24  5  0                   1        NaN   
149  San Diego State     6    25  4  0                   1        NaN   
150             Team  Week  Rank  W  L  Winning Percentage  Opp. Rank   

     Opp. P5  Home  Result  Points Scored  Points Against  Margin  \
0          1     0       W             44              13      31   
1          0     1       W             40              35       5   
2          1     0       L              3              10      -7   
3          1     0       W             45              31      14   
4          1     0       W             10               3       7   
..       ...   ...     ...            ...             ...     ...   
146      NaN   NaN     NaN            NaN             NaN     NaN   
147      NaN   NaN     NaN            NaN             NaN     NaN   
148      NaN   NaN     NaN            NaN             NaN     NaN   
149      NaN   NaN     NaN            NaN             NaN     NaN   
150  Opp. P5  Home  Result  Points Scored  Points Against  Margin   

     Next Week Rank  Movement  
0                 1         0  
1                 4        -2  
2                 6        -3  
3                 3         1  
4                 2         3  
..              ...       ...  
146             NaN       NaN  
147             NaN       NaN  
148             NaN       NaN  
149             NaN       NaN  
150  Next Week Rank  Movement  

[151 rows x 15 columns]

In [3]:
#Drop footer
df.drop(index = (df.index.stop - 1), inplace = True)

#Drop null columns (teams with byes)
df.dropna(inplace = True)

#Set week
for index, row in df.iterrows():
    if row['Week'] >= next_week:
        df.drop(index = index, inplace = True)

In [4]:
#Make W/L binary values
for index, row in df.iterrows():
    if row['Result'] == 'W':
        df.at[index, 'Result'] = 1
    else: 
        df.at[index, 'Result'] = 0

In [5]:
df = df.drop(columns = ["Movement", "Team"])

In [6]:
for name, values in df.iteritems():
    df[name] = pd.to_numeric(values)
    
df.dtypes

Week                    int64
Rank                    int64
W                       int64
L                       int64
Winning Percentage    float64
Opp. Rank               int64
Opp. P5                 int64
Home                    int64
Result                  int64
Points Scored           int64
Points Against          int64
Margin                  int64
Next Week Rank          int64
dtype: object

In [7]:
#Group rankings to categories
for index, row in df.iterrows():
    if row['Next Week Rank'] == 26:
        df.at[index, 'Next Week Rank Category'] = 0
    elif row['Next Week Rank'] < row['Rank']:
        df.at[index, 'Next Week Rank Category'] = 1
    elif row['Next Week Rank'] > row['Rank']:
        df.at[index, 'Next Week Rank Category'] = 2
    elif row['Next Week Rank'] == row['Rank']:
        df.at[index, 'Next Week Rank Category'] = 3        

In [8]:
X = df.drop(columns = ['Next Week Rank', 'Next Week Rank Category'])
y = df['Next Week Rank Category']

In [9]:
y

0     3.0
1     2.0
2     2.0
3     1.0
4     1.0
     ... 
95    0.0
96    1.0
97    1.0
98    1.0
99    0.0
Name: Next Week Rank Category, Length: 96, dtype: float64

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 8)
X_train.shape

(72, 12)

In [11]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [12]:
#Decision Tree classifier
from sklearn import tree
dtc = tree.DecisionTreeClassifier(random_state = 8)
dtc.fit(X_train, y_train)
dtc.score(X_test, y_test)

0.5833333333333334

In [13]:
#Scaled Decision Tree classifier
from sklearn import tree
scaled_dtc = tree.DecisionTreeClassifier(random_state = 8)
scaled_dtc.fit(X_train_scaled, y_train)
scaled_dtc.score(X_test_scaled, y_test)

0.5833333333333334

In [14]:
#Random Forest classifier
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state = 8)
rfc.fit(X_train, y_train)
rfc.score(X_test, y_test)

0.625

In [15]:
#Scaled Random Forest classifier
from sklearn.ensemble import RandomForestClassifier
scaled_rfc = RandomForestClassifier(random_state = 8)
scaled_rfc.fit(X_train_scaled, y_train)
scaled_rfc.score(X_test_scaled, y_test)

0.625

In [16]:
#Support Vector Machine classifier
from sklearn.svm import SVC
svm = SVC(random_state = 8)
svm.fit(X_train, y_train)
svm.score(X_test, y_test)

0.7083333333333334

In [17]:
#Scaled SVMC
svm = SVC(random_state = 8)
svm.fit(X_train_scaled, y_train)
svm.score(X_test_scaled, y_test)

0.625